In [1]:
using Pkg
Pkg.add(["JuMP", "Downloads", "Distances","HiGHS","Random","Statistics","Printf","Zygote","SparseArrays","LIBSVMdata"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed HiGHS_jll ─────────── v1.11.0+1
   Installed ZygoteRules ───────── v0.2.7
   Installed ProgressBars ──────── v1.5.1
   Installed Distances ─────────── v0.10.12
   Installed IRTools ───────────── v0.4.15
   Installed RealDot ───────────── v0.1.0
   Installed MutableArithmetics ── v1.6.4
   Installed MathOptInterface ──── v1.43.0
   Installed CodecBzip2 ────────── v0.8.5
   Installed StructTypes ───────── v1.11.0
   Installed JSON3 ─────────────── v1.14.3
   Installed BenchmarkTools ────── v1.6.0
   Installed Zygote ────────────── v0.7.10
   Installed ChainRules ────────── v1.72.5
   Installed SparseInverseSubset ─ v0.1.2
   Installed MathOptIIS ────────── v0.1.1
   Installed HiGHS ─────────────── v1.19.0
   Installed JuMP ──────────────── v1.28.0
   Installed LIBSVMdata ────────── v0.1.4
    Updating `~/.julia/environments/v1.11/Project.toml`
  [b4f34e82] + Distances v0.10.12
  [87d

In [ ]:
using JuMP, HiGHS    #a1a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a1a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a1a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a1a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                             ┫ 0/1.6k [00:00<00:00, -0s/it]
0.1%┣                                         ┫ 1/1.6k [00:00<Inf:Inf, InfGs/it]
16.0%┣██████▋                                  ┫ 257/1.6k [00:01<00:03, 461it/s]
100.0%┣██████████████████████████████████████┫ 1.6k/1.6k [00:01<00:00, 2.7kit/s]
100.0%┣██████████████████████████████████████┫ 1.6k/1.6k [00:01<00:00, 2.7kit/s]
[ Info: Data loaded: 1605 samples, 123 features



🚀 Starting VDCG...
Initial solution: f(x) = 0.950277


...

  New point found: f(x) = 0.936871


[ Info: Interior point LP: τ = -77.74566753, Status = Interior
[ Info: Interior point LP: τ = -189.07573742, Status = Interior
[ Info: Cell 57: New point not added, obj = 0.937968, min distance = 0.0, better = false


  New point found: f(x) = 0.933626
  ✅ Level 8 completed in 20.4761 sec | Best objective so far: 0.930190


[ Info: Interior point LP: τ = -38.87286661, Status = Interior


 60.414592 seconds (35.75 M allocations: 28.031 GiB, 8.08% gc time, 22.49% compilation time)

✅ VDCG Best Objective: 0.930190
✅ VDCG Total Time: 86.0436 sec, Total Points: 134
📊 Running Standard CG...
📊 Standard CG Objective: 0.950277
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.950277, Time so far: 0.2857 sec, Points: 1
  Seed 24: f(x) = 0.940751, Time so far: 0.5751 sec, Points: 2
  Seed 25: f(x) = 0.937425, Time so far: 0.8613 sec, Points: 3
  Seed 26: f(x) = 0.936839, Time so far: 1.1492 sec, Points: 4
  Seed 27: f(x) = 0.941972, Time so far: 1.4266 sec, Points: 5
  Seed 28: f(x) = 0.936335, Time so far: 1.7306 sec, Points: 6
  Seed 29: f(x) = 1.168775, Time so far: 2.0256 sec, Points: 7
  Seed 30: f(x) = 0.942528, Time so far: 2.3102 sec, Points: 8
  Seed 31: f(x) = 0.936355, Time so far: 2.6018 sec, Points: 9
  Seed 32: f(x) = 0.934619, Time so far: 2.9514 sec, Points: 10
  Seed 33: f(x) = 0.939360, Time so far: 3.2703 sec, Points: 11
  Seed 34: f(x) = 0.957168,

In [ ]:
using JuMP, HiGHS    #a2a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a2a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a2a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a2a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                             ┫ 0/2.3k [00:00<00:00, -0s/it]



🚀 Starting VDCG...
Initial solution: f(x) = 0.956188


...

  New point found: f(x) = 0.945655


[ Info: Interior point LP: τ = -40.39458108, Status = Interior


  New point found: f(x) = 1.000000
  New point found: f(x) = 0.945788


[ Info: Interior point LP: τ = -22.20656462, Status = Interior
[ Info: Interior point LP: τ = -15.0433551, Status = Interior
[ Info: Cell 43: New point not added, obj = 1.0, min distance = 0.0, better = false
[ Info: Interior point LP: τ = -23.56200839, Status = Interior
[ Info: Interior point LP: τ = -190.62550079, Status = Interior
[ Info: Cell 45: New point not added, obj = 0.967018, min distance = 0.0, better = false


  ✅ Level 8 completed in 21.6030 sec | Best objective so far: 0.942411
 55.489279 seconds (20.38 M allocations: 29.877 GiB, 11.31% gc time, 2.76% compilation time: 13% of which was recompilation)

✅ VDCG Best Objective: 0.942411
✅ VDCG Total Time: 95.8796 sec, Total Points: 114
📊 Running Standard CG...
📊 Standard CG Objective: 0.956188
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.956188, Time so far: 0.4332 sec, Points: 1
  Seed 24: f(x) = 0.948558, Time so far: 0.8886 sec, Points: 2
  Seed 25: f(x) = 0.946616, Time so far: 1.3289 sec, Points: 3
  Seed 26: f(x) = 0.945308, Time so far: 1.7615 sec, Points: 4
  Seed 27: f(x) = 0.949792, Time so far: 2.2321 sec, Points: 5
  Seed 28: f(x) = 0.945766, Time so far: 2.6550 sec, Points: 6
  Seed 29: f(x) = 0.950026, Time so far: 3.1185 sec, Points: 7
  Seed 30: f(x) = 0.953997, Time so far: 3.7084 sec, Points: 8
  Seed 31: f(x) = 0.946563, Time so far: 4.4354 sec, Points: 9
  Seed 32: f(x) = 0.945797, Time so far: 5.0945 sec,

In [ ]:
using JuMP, HiGHS    #a3a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a3a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a3a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a3a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                             ┫ 0/3.2k [00:00<00:00, -0s/it]
49.2%┣██████████████████▊                   ┫ 1.6k/3.2k [00:00<00:00, 31.1kit/s]



🚀 Starting VDCG...
Initial solution: f(x) = 0.956656


...

  New point found: f(x) = 0.946273


[ Info: Interior point LP: τ = -35.21023777, Status = Interior


  New point found: f(x) = 0.947985


[ Info: Interior point LP: τ = -155.33508667, Status = Interior
[ Info: Interior point LP: τ = -604.30986688, Status = Interior
[ Info: Cell 46: New point not added, obj = 0.945798, min distance = 0.0, better = false
[ Info: Interior point LP: τ = -119.14444099, Status = Interior
[ Info: Cell 47: New point not added, obj = 0.94518, min distance = 0.0, better = false


  ✅ Level 8 completed in 32.0528 sec | Best objective so far: 0.942901
 79.450919 seconds (21.15 M allocations: 42.516 GiB, 9.99% gc time, 2.09% compilation time: 13% of which was recompilation)

✅ VDCG Best Objective: 0.942901
✅ VDCG Total Time: 139.5299 sec, Total Points: 116
📊 Running Standard CG...
📊 Standard CG Objective: 0.956656
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.956656, Time so far: 0.5825 sec, Points: 1
  Seed 24: f(x) = 0.949314, Time so far: 1.1900 sec, Points: 2
  Seed 25: f(x) = 0.947292, Time so far: 1.8120 sec, Points: 3
  Seed 26: f(x) = 0.945849, Time so far: 2.4309 sec, Points: 4
  Seed 27: f(x) = 1.340922, Time so far: 3.0311 sec, Points: 5
  Seed 28: f(x) = 0.945774, Time so far: 3.5859 sec, Points: 6
  Seed 29: f(x) = 0.948422, Time so far: 4.1755 sec, Points: 7
  Seed 30: f(x) = 0.994024, Time so far: 4.7581 sec, Points: 8
  Seed 31: f(x) = 0.945762, Time so far: 5.3342 sec, Points: 9
  Seed 32: f(x) = 0.944996, Time so far: 5.8984 sec,

In [ ]:
using JuMP, HiGHS    #a4a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a4a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a4a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a4a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                             ┫ 0/4.8k [00:00<00:00, -0s/it]
24.0%┣█████████▏                            ┫ 1.1k/4.8k [00:00<00:00, 22.8kit/s]
44.5%┣█████████████████                     ┫ 2.1k/4.8k [00:00<00:00, 20.7kit/s]



🚀 Starting VDCG...
Initial solution: f(x) = 0.955828


...

  New point found: f(x) = 0.944251


[ Info: Interior point LP: τ = -602.49096979, Status = Interior


  New point found: f(x) = 0.942493


[ Info: Interior point LP: τ = -10.15104035, Status = Interior
[ Info: Interior point LP: τ = -136.84102495, Status = Interior
[ Info: Cell 58: New point not added, obj = 0.942964, min distance = 0.0, better = false


  ✅ Level 8 completed in 57.2380 sec | Best objective so far: 0.940524
128.596532 seconds (23.28 M allocations: 68.438 GiB, 9.58% gc time, 1.40% compilation time: 11% of which was recompilation)

✅ VDCG Best Objective: 0.940524
✅ VDCG Total Time: 226.3476 sec, Total Points: 132
📊 Running Standard CG...
📊 Standard CG Objective: 0.955828
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.955828, Time so far: 1.3941 sec, Points: 1
  Seed 24: f(x) = 0.946862, Time so far: 2.8777 sec, Points: 2
  Seed 25: f(x) = 0.944747, Time so far: 3.9064 sec, Points: 3
  Seed 26: f(x) = 0.943847, Time so far: 4.8551 sec, Points: 4
  Seed 27: f(x) = 1.310069, Time so far: 5.7998 sec, Points: 5
  Seed 28: f(x) = 0.943906, Time so far: 6.7433 sec, Points: 6
  Seed 29: f(x) = 1.172646, Time so far: 7.6683 sec, Points: 7
  Seed 30: f(x) = 0.992443, Time so far: 8.6234 sec, Points: 8
  Seed 31: f(x) = 0.944017, Time so far: 9.5607 sec, Points: 9
  Seed 32: f(x) = 0.943064, Time so far: 10.4894 sec

In [2]:
using JuMP, HiGHS    #a5a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a5a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a5a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a5a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                             ┫ 0/6.4k [00:00<00:00, -0s/it]
0.0%┣                                         ┫ 1/6.4k [00:00<Inf:Inf, InfGs/it]
34.9%┣█████████████▋                         ┫ 2.2k/6.4k [00:00<00:01, 5.2kit/s]
68.4%┣██████████████████████████▊            ┫ 4.4k/6.4k [00:00<00:00, 9.1kit/s]
78.7%┣██████████████████████████████▊        ┫ 5.1k/6.4k [00:01<00:00, 9.5kit/s]
100.0%┣█████████████████████████████████████┫ 6.4k/6.4k [00:01<00:00, 11.4kit/s]
100.0%┣█████████████████████████████████████┫ 6.4k/6.4k [00:01<00:00, 11.4kit/s]
[ Info: Data loaded: 6414 samples, 123 features



🚀 Starting VDCG...
Initial solution: f(x) = 0.958658


...

  New point found: f(x) = 0.946155


[ Info: Interior point LP: τ = -59.79414497, Status = Interior


  New point found: f(x) = 1.005602
  ✅ Level 8 completed in 46.5073 sec | Best objective so far: 0.943594


[ Info: Interior point LP: τ = -662.32606271, Status = Interior


120.078793 seconds (27.93 M allocations: 65.845 GiB, 6.81% gc time, 9.63% compilation time)

✅ VDCG Best Objective: 0.943594
✅ VDCG Total Time: 196.0998 sec, Total Points: 105
📊 Running Standard CG...
📊 Standard CG Objective: 0.958658
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.958658, Time so far: 1.7017 sec, Points: 1
  Seed 24: f(x) = 0.948882, Time so far: 2.8331 sec, Points: 2
  Seed 25: f(x) = 0.947474, Time so far: 3.9556 sec, Points: 3
  Seed 26: f(x) = 0.947078, Time so far: 5.0646 sec, Points: 4
  Seed 27: f(x) = 0.953103, Time so far: 6.1930 sec, Points: 5
  Seed 28: f(x) = 0.947001, Time so far: 7.2843 sec, Points: 6
  Seed 29: f(x) = 0.950725, Time so far: 8.3889 sec, Points: 7
  Seed 30: f(x) = 0.959785, Time so far: 9.4822 sec, Points: 8
  Seed 31: f(x) = 0.946974, Time so far: 10.5764 sec, Points: 9
  Seed 32: f(x) = 0.946475, Time so far: 11.6823 sec, Points: 10
  Seed 33: f(x) = 0.948720, Time so far: 13.3760 sec, Points: 11
  Seed 34: f(x) = 0.9664

In [3]:
using JuMP, HiGHS    #a6a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a6a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a6a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a6a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                            ┫ 0/11.2k [00:00<00:00, -0s/it]
17.1%┣██████▎                              ┫ 1.9k/11.2k [00:00<00:00, 38.0kit/s]
30.4%┣███████████▎                         ┫ 3.4k/11.2k [00:00<00:00, 33.8kit/s]
46.9%┣█████████████████▍                   ┫ 5.3k/11.2k [00:00<00:00, 34.8kit/s]
52.4%┣███████████████████▍                 ┫ 5.9k/11.2k [00:00<00:00, 29.2kit/s]



🚀 Starting VDCG...
Initial solution: f(x) = 0.962249


...

  New point found: f(x) = 0.950230


[ Info: Interior point LP: τ = -83.06561978, Status = Interior
[ Info: Interior point LP: τ = -15.01459418, Status = Interior
[ Info: Cell 44: New point not added, obj = 0.960685, min distance = 0.0, better = false
[ Info: Interior point LP: τ = -111.54230138, Status = Interior
[ Info: Cell 45: New point not added, obj = 0.957421, min distance = 0.0, better = false


  New point found: f(x) = 0.954475
  ✅ Level 8 completed in 97.9449 sec | Best objective so far: 0.948269


[ Info: Interior point LP: τ = -134.76867311, Status = Interior


243.731390 seconds (21.46 M allocations: 143.529 GiB, 7.42% gc time, 0.57% compilation time: 8% of which was recompilation)

✅ VDCG Best Objective: 0.948269
✅ VDCG Total Time: 432.1264 sec, Total Points: 114
📊 Running Standard CG...
📊 Standard CG Objective: 0.962249
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.962249, Time so far: 1.9273 sec, Points: 1
  Seed 24: f(x) = 0.952247, Time so far: 3.8561 sec, Points: 2
  Seed 25: f(x) = 0.951239, Time so far: 5.7832 sec, Points: 3
  Seed 26: f(x) = 0.950755, Time so far: 7.8299 sec, Points: 4
  Seed 27: f(x) = 0.952708, Time so far: 10.6621 sec, Points: 5
  Seed 28: f(x) = 0.950469, Time so far: 12.5926 sec, Points: 6
  Seed 29: f(x) = 1.186022, Time so far: 14.5155 sec, Points: 7
  Seed 30: f(x) = 0.995092, Time so far: 16.4597 sec, Points: 8
  Seed 31: f(x) = 0.950502, Time so far: 18.3977 sec, Points: 9
  Seed 32: f(x) = 0.949645, Time so far: 20.3212 sec, Points: 10
  Seed 33: f(x) = 0.952808, Time so far: 23.2025 sec,

In [4]:
using JuMP, HiGHS    #a7a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a7a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a7a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a7a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                            ┫ 0/16.1k [00:00<00:00, -0s/it]
12.1%┣████▌                                ┫ 2.0k/16.1k [00:00<00:00, 38.9kit/s]
23.2%┣████████▋                            ┫ 3.7k/16.1k [00:00<00:00, 37.0kit/s]
33.5%┣████████████▍                        ┫ 5.4k/16.1k [00:00<00:00, 35.7kit/s]
40.7%┣███████████████                      ┫ 6.6k/16.1k [00:00<00:00, 32.5kit/s]
53.9%┣████████████████████                 ┫ 8.7k/16.1k [00:00<00:00, 34.4kit/s]
64.5%┣███████████████████████▎            ┫ 10.4k/16.1k [00:00<00:00, 34.4kit/s]
75.0%┣███████████████████████████         ┫ 12.1k/16.1k [00:00<00:00, 34.3kit/s]
85.3%┣██████████████████████████████▊     ┫ 13.7k/16.1k [00:00<00:00, 31.8kit/s]



🚀 Starting VDCG...
Initial solution: f(x) = 0.962249


...

  New point found: f(x) = 0.949767


[ Info: Interior point LP: τ = -17.29381617, Status = Interior


  New point found: f(x) = 0.949381
  ✅ Level 8 completed in 139.4150 sec | Best objective so far: 0.947613


[ Info: Interior point LP: τ = -34.79464187, Status = Interior


308.288156 seconds (19.35 M allocations: 181.074 GiB, 7.38% gc time, 0.43% compilation time: 13% of which was recompilation)

✅ VDCG Best Objective: 0.947613
✅ VDCG Total Time: 546.1385 sec, Total Points: 123
📊 Running Standard CG...
📊 Standard CG Objective: 0.962249
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.962249, Time so far: 3.2142 sec, Points: 1
  Seed 24: f(x) = 0.951342, Time so far: 6.5886 sec, Points: 2
  Seed 25: f(x) = 0.950568, Time so far: 9.4187 sec, Points: 3
  Seed 26: f(x) = 0.949695, Time so far: 12.2819 sec, Points: 4
  Seed 27: f(x) = 0.952048, Time so far: 15.2194 sec, Points: 5
  Seed 28: f(x) = 0.949920, Time so far: 18.9020 sec, Points: 6
  Seed 29: f(x) = 1.183626, Time so far: 21.6841 sec, Points: 7
  Seed 30: f(x) = 0.995734, Time so far: 24.4655 sec, Points: 8
  Seed 31: f(x) = 0.949906, Time so far: 27.2594 sec, Points: 9
  Seed 32: f(x) = 0.948999, Time so far: 31.0704 sec, Points: 10
  Seed 33: f(x) = 0.951541, Time so far: 33.8737 se

In [5]:
using JuMP, HiGHS    #a8a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a8a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a8a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a8a'...


* Couldn't find host www.csie.ntu.edu.tw in the .netrc file; using defaults
* Host www.csie.ntu.edu.tw:443 was resolved.
* IPv6: (none)
* IPv4: 140.112.30.26
*   Trying 140.112.30.26:443...
* Connected to www.csie.ntu.edu.tw (140.112.30.26) port 443
* mbedTLS: Connecting to www.csie.ntu.edu.tw:443
* mbedTLS: Set min SSL version to TLS 1.0
* ALPN: curl offers h2,http/1.1
* mbedTLS: Handshake complete, cipher is TLS-ECDHE-RSA-WITH-AES-128-GCM-SHA256
* Dumping cert info: * cert. version     : 3
* serial number     : 47:E8:00:00:00:07:87:FE:49:35:DC:01:F3:CD:23:5B
* issuer name       : C=TW, O=TAIWAN-CA, CN=TWCA Secure SSL Certification Authority
* subject name      : C=TW, ST=Taiwan, L=Taipei, O=National Taiwan University, CN=*.csie.ntu.edu.tw
* issued  on        : 2024-10-16 09:35:59
* expires on        : 2025-11-03 15:59:59
* signed using      : RSA with SHA-256
* RSA key size      : 2048 bits
* basic constraints : CA=false
* subject alt name  :
*     dNSName : *.csie.ntu.edu.tw
*     d

Loading the dataset...


* Connection #0 to host www.csie.ntu.edu.tw left intact
0.0%┣                                            ┫ 0/22.7k [00:00<00:00, -0s/it]
9.6%┣███▊                                  ┫ 2.2k/22.7k [00:00<00:00, 43.5kit/s]
14.1%┣█████▎                               ┫ 3.2k/22.7k [00:00<00:01, 31.8kit/s]
24.1%┣█████████                            ┫ 5.5k/22.7k [00:00<00:00, 36.3kit/s]
30.9%┣███████████▍                         ┫ 7.0k/22.7k [00:00<00:00, 34.6kit/s]
40.9%┣███████████████▏                     ┫ 9.3k/22.7k [00:00<00:00, 36.7kit/s]
50.4%┣██████████████████▏                 ┫ 11.4k/22.7k [00:00<00:00, 37.7kit/s]
58.3%┣█████████████████████               ┫ 13.2k/22.7k [00:00<00:00, 35.5kit/s]
68.2%┣████████████████████████▌           ┫ 15.5k/22.7k [00:00<00:00, 36.6kit/s]
78.0%┣████████████████████████████        ┫ 17.7k/22.7k [00:00<00:00, 37.4kit/s]
88.2%┣███████████████████████████████▊    ┫ 20.0k/22.7k [00:01<00:00, 38.2kit/s]



🚀 Starting VDCG...
Initial solution: f(x) = 0.963102


...

  New point found: f(x) = 1.049216


[ Info: Interior point LP: τ = -28.95382435, Status = Interior


  New point found: f(x) = 0.993135
  ✅ Level 8 completed in 210.2198 sec | Best objective so far: 0.948087


[ Info: Interior point LP: τ = -53.40810146, Status = Interior


507.847619 seconds (22.47 M allocations: 301.116 GiB, 7.16% gc time, 0.25% compilation time: 14% of which was recompilation)

✅ VDCG Best Objective: 0.948087
✅ VDCG Total Time: 904.5734 sec, Total Points: 122
📊 Running Standard CG...
📊 Standard CG Objective: 0.963102
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.963102, Time so far: 4.9104 sec, Points: 1
  Seed 24: f(x) = 0.952292, Time so far: 8.7710 sec, Points: 2
  Seed 25: f(x) = 0.951167, Time so far: 12.6317 sec, Points: 3
  Seed 26: f(x) = 0.950255, Time so far: 17.9269 sec, Points: 4
  Seed 27: f(x) = 0.953081, Time so far: 21.8506 sec, Points: 5
  Seed 28: f(x) = 0.950604, Time so far: 25.8126 sec, Points: 6
  Seed 29: f(x) = 1.184993, Time so far: 30.7197 sec, Points: 7
  Seed 30: f(x) = 0.995973, Time so far: 34.6503 sec, Points: 8
  Seed 31: f(x) = 0.950593, Time so far: 38.5608 sec, Points: 9
  Seed 32: f(x) = 0.949781, Time so far: 43.4889 sec, Points: 10
  Seed 33: f(x) = 0.952354, Time so far: 47.4073 s

In [6]:
using JuMP, HiGHS    #a9a
using LinearAlgebra
using Zygote
using LIBSVMdata
using Printf
using Random
using Statistics
using SparseArrays

# Set random seed for reproducibility
Random.seed!(23)

# ================== Configuration ================== #
const MAX_LEVELS = 8
const TOL_UNIQUE = 1e-3
const TOL_CONVERGENCE = 1e-6
const TAU_L1 = 50.0  # L1-norm ball radius
const tau = 50.0     # L1-norm ball radius

# ================== Global Data ================== #
A = spzeros(1, 1)
y = zeros(1)
n = 1

# Sigmoid function
sigmoid(z) = 1 / (1 + exp(-z))

# Objective and gradient
f(x) = mean((y .- sigmoid.(A * x)).^2)
grad_f(x) = Zygote.gradient(f, x)[1]

# Linear Minimization Oracle for the L1-ball ||x||_1 ≤ τ
function lmo(g)
    i = argmax(abs.(g))
    v = zeros(n)
    v[i] = -TAU_L1 * sign(g[i])
    return v
end

# Check if a point is new
function is_new_point(x, points, tol=TOL_UNIQUE)
    all(norm(x - p) >= tol for p in points)
end

# ================== Adaptive Frank-Wolfe Algorithm ================== #
function conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0; max_iter=1000, epsilon=TOL_CONVERGENCE, delta=1e-10, beta=2, gamma=0.5)
    x_prev = copy(x0)
    x_curr = copy(x0)
    values = [f(x_curr)]
    times = [0.0]
    gaps = Float64[]
    L_ks = Float64[]
    steps = Float64[]
    backtrack_counts = Int[]
    gamma_history = [gamma]
    k = 0
    prev_grad = grad_f(x_prev)
    current_f = f(x_curr)
    recent_backtracks = Int[]

    while k < max_iter
        start = time()
        current_grad = grad_f(x_curr)
        v = lmo(current_grad)
        d = v - x_curr
        normd2 = dot(d, d)
        gap = -dot(current_grad, d)

        if gap <= epsilon
            push!(times, time() - start)
            push!(gaps, gap)
            break
        end

        if k == 0
            d0 = ones(length(x0)) / sqrt(length(x0))
            x_temp = x0 + 1e-3 * d0
            L_k = gamma * (norm(grad_f(x0) - grad_f(x_temp)) / (1e-3 * norm(d0)) + delta)
        else
            grad_diff = norm(current_grad - prev_grad)
            x_diff = norm(x_curr - x_prev)
            L_k = gamma * (grad_diff / x_diff + delta)
        end
        Lknormd2 = L_k * normd2
        t_k = min(gap / Lknormd2, 1.0)
        i = 0
        while true
            x_new = x_curr + t_k * d
            new_f = f(x_new)
            if current_f - new_f >= t_k * gap - (Lknormd2 / 2) * t_k^2
                x_prev = copy(x_curr)
                x_curr = x_new
                push!(backtrack_counts, i)
                push!(recent_backtracks, i)
                break
            else
                L_k *= beta
                Lknormd2 = L_k * normd2
                t_k = min(gap / Lknormd2, 1.0)
                i += 1
            end
        end

        if k % 10 == 0 && k > 0
            total_backtracks = sum(recent_backtracks)
            if total_backtracks == 0
                gamma = max(1e-4, gamma * 0.9)
            elseif total_backtracks > 10
                gamma = min(1.0, gamma * 1.1)
            end
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        elseif k % 10 == 0
            push!(gamma_history, gamma)
            recent_backtracks = Int[]
        else
            push!(recent_backtracks, i)
        end
        k += 1
        iteration_time = time() - start

        current_f = f(x_curr)
        prev_grad = current_grad
        push!(gaps, gap)
        push!(steps, t_k)
        push!(values, current_f)
        push!(times, iteration_time)
        push!(L_ks, L_k)

        if k % 100 == 0
            # println("k=$k, gap=$(round(gap, digits=4)), t_k=$(round(t_k, digits=4)), L_k=$(round(L_ks[end], digits=6)), Time=$(round(times[end], digits=4)), f=$(round(values[end], digits=6))")
        end
    end
    total_time = sum(times)
    return (x_curr, values, times, gaps, L_ks, backtrack_counts, steps, gamma_history, total_time)
end

# ================== Interior Point via Slack (Paper's Method) ================== #
function find_interior_point(p_i, sites)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, xp[1:n] >= 0)
    @variable(model, xn[1:n] >= 0)
    @variable(model, τ)

    # L1 constraint: ||x||_1 ≤ TAU_L1
    @constraint(model, sum(xp) + sum(xn) <= TAU_L1)

    # Reconstruct x = xp - xn
    x = xp - xn

    # Voronoi cell constraints with slack
    for j in 1:length(sites)
        if sites[j] ≈ p_i
            continue
        end
        p_j = sites[j]
        a = p_j - p_i
        b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
        @constraint(model, dot(a, x) <= b_val + τ)
    end
    @objective(model, Min, τ)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        x_sol = value.(xp) - value.(xn)
        τ_sol = value(τ)
        @info "Interior point LP: τ = $(round(τ_sol, digits=8)), Status = $(τ_sol < -1e-8 ? "Interior" : "Boundary")"
        return τ_sol < -1e-8 ? (x_sol, "Interior") : (x_sol, "Boundary")
    else
        @warn "Interior point LP failed: Infeasible or unbounded"
        if length(sites) == 1
            @info "Returning barycenter as fallback"
            return zeros(n), "Interior"
        end
        return nothing, "Infeasible"
    end
end

# ================== Voronoi Partitioning ================== #
function create_voronoi_partitions(sites)
    base_A = vcat(
        Matrix{Float64}(I, n, n),      # x ≥ -TAU_L1
        -Matrix{Float64}(I, n, n)      # x ≤ TAU_L1
    )
    base_b = vcat(
        TAU_L1 * ones(n),
        TAU_L1 * ones(n)
    )
    partitions = []
    K = length(sites)
    if K == 1
        push!(partitions, (base_A, base_b))
        return partitions
    end
    for i in 1:K
        A_i = copy(base_A)
        b_i = copy(base_b)
        p_i = sites[i]
        for j in 1:K
            i == j && continue
            p_j = sites[j]
            a = (p_j - p_i)'
            b_val = 0.5 * dot(p_j + p_i, p_j - p_i)
            A_i = vcat(A_i, a)
            b_i = vcat(b_i, b_val)
        end
        push!(partitions, (A_i, b_i))
    end
    return partitions
end

# ================== Main VDCG Algorithm ================== #
function voronoi_conditional_gradient(f, grad_f, lmo, x0)
    x_init, _, cg_times, _, _, _, _, _, init_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    total_time = init_time
    sites = [x0, x_init]
    num_points = 1
    archive = [(x_init, f(x_init))]
    best_so_far = f(x_init)
    @printf("Initial solution: f(x) = %.6f\n", f(x_init))
    @info "Distance between x0 and x_init: $(norm(x0 - x_init))"

    for level in 1:MAX_LEVELS
        start_time = time()
        @printf("→ VDCG Level %d: %d sites, Current best f(x) = %.6f\n", level, length(sites), best_so_far)
        partitions = create_voronoi_partitions(sites)
        new_sites = []
        for (i, (A_cell, b_cell)) in enumerate(partitions)
            x_inner, status = find_interior_point(sites[i], sites)
            if x_inner === nothing || status == "Infeasible"
                @info "Skipping cell $i: $status"
                continue
            end
            x_stat, _, cg_times, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_inner)
            total_time += cg_time
            num_points += 1
            obj = f(x_stat)
            all_points = [s for (s, _) in archive]
            if is_new_point(x_stat, all_points) && is_new_point(x_stat, new_sites)
                push!(new_sites, x_stat)
                push!(archive, (x_stat, obj))
                @printf("  New point found: f(x) = %.6f\n", obj)
                best_so_far = min(best_so_far, obj)
            else
                min_dist = minimum(norm(x_stat - p) for p in all_points)
                @info "Cell $i: New point not added, obj = $(round(obj, digits=6)), min distance = $(round(min_dist, digits=6)), better = $(obj < best_so_far)"
            end
        end
        elapsed = time() - start_time
        total_time += elapsed
        if isempty(new_sites)
            @printf("→ No new points found. Terminating at level %d.\n", level)
            @printf("  🕒 Level %d runtime: %.4f sec | Best objective: %.6f\n", level, elapsed, best_so_far)
            break
        else
            append!(sites, new_sites)
            @printf("  ✅ Level %d completed in %.4f sec | Best objective so far: %.6f\n", level, elapsed, best_so_far)
        end
    end
    best_idx = argmin([obj for (_, obj) in archive])
    x_best, f_best = archive[best_idx]
    return x_best, f_best, archive, sites, total_time, num_points
end

# ================== Load Problem Data ================== #
function load_libsvm_data()
    data_name = "a9a"
    @info "Loading LIBSVM dataset '$data_name'..."
    global A, y, n
    A_loaded, y_loaded = load_dataset(data_name, dense=false, replace=false, verbose=true)
    A = A_loaded
    y = Float64.(y_loaded)
    m, n = size(A)
    # Map labels: {-1,1} → {0,1} for sigmoid output (optional)
    # y = (y .+ 1) ./ 2
    Random.seed!(23)
    V = vcat(tau * I(n), -tau * I(n))
    global x0 = V[rand(1:size(V, 1)), :]
    @info "Data loaded: $m samples, $n features"
    @info "Initial point x0 set on L1-ball boundary"
end

# ================== Run Experiment ================== #
function main()
    # Load data for sigmoid regression
    load_libsvm_data()

    # Run VDCG
    @printf("\n🚀 Starting VDCG...\n")
    @time x_vdcg, f_vdcg, archive, sites, vdcg_time, vdcg_points = voronoi_conditional_gradient(f, grad_f, lmo, x0)
    @printf("\n✅ VDCG Best Objective: %.6f\n", f_vdcg)
    @printf("✅ VDCG Total Time: %.4f sec, Total Points: %d\n", vdcg_time, vdcg_points)

    # Standard CG
    @printf("📊 Running Standard CG...\n")
    x_std, _, _, _, _, _, _, _, std_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x0)
    f_std = f(x_std)
    @printf("📊 Standard CG Objective: %.6f\n", f_std)

    # Multi-start CG (limited by VDCG time)
    @printf("🔍 Running Multi-start CG (time-limited)...\n")
    f_multi = Inf
    total_multi_time = 0.0
    points_evaluated = 0
    seed = 23
    while total_multi_time < vdcg_time
        Random.seed!(seed)
        start = time()

        # For the first iteration (seed 23), use the same x0 as standard CG
        if seed == 23
            x_rand = copy(x0)
        else
            V = vcat(tau * I(n), -tau * I(n))
            x_rand = V[rand(1:size(V, 1)), :]
        end

        x, _, _, _, _, _, _, _, cg_time = conditional_gradient_adjustable_scaling(f, grad_f, lmo, x_rand)
        total_multi_time += time() - start
        points_evaluated += 1
        f_current = f(x)
        f_multi = min(f_multi, f_current)

        @printf("  Seed %d: f(x) = %.6f, Time so far: %.4f sec, Points: %d\n", seed, f_current, total_multi_time, points_evaluated)
        seed += 1
    end
    @printf("🔍 Multi-start CG Best: %.6f (Time: %.4f sec, Points: %d)\n", f_multi, total_multi_time, points_evaluated)

    # Summary Table
    @printf("\n📋 Summary of Results for Sigmoid Regression (a9a)\n")
    @printf("┌──────────────────────┬─────────────────┬──────────────┬─────────────────┐\n")
    @printf("│ %-20s │ %-15s │ %-12s │ %-15s │\n", "Method", "Objective Value", "Runtime (sec)", "Points Evaluated")
    @printf("├──────────────────────┼─────────────────┼──────────────┼─────────────────┤\n")
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "VDCG", f_vdcg, vdcg_time, vdcg_points)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Standard CG", f_std, std_time, 1)
    @printf("│ %-20s │ %-15.6f │ %-12.4f │ %-15d │\n", "Multi-start CG", f_multi, total_multi_time, points_evaluated)
    @printf("└──────────────────────┴─────────────────┴──────────────┴─────────────────┘\n")
end

# Run everything
main()

[ Info: Loading LIBSVM dataset 'a9a'...


The data a9a was already downloaded
Loading the dataset...


0.0%┣                                            ┫ 0/22.7k [00:00<00:00, -0s/it]
9.8%┣███▊                                  ┫ 2.2k/22.7k [00:00<00:00, 44.4kit/s]
15.0%┣█████▌                               ┫ 3.4k/22.7k [00:00<00:01, 33.7kit/s]
23.4%┣████████▋                            ┫ 5.3k/22.7k [00:00<00:00, 35.1kit/s]
29.4%┣██████████▉                          ┫ 6.7k/22.7k [00:00<00:00, 33.1kit/s]
39.2%┣██████████████▌                      ┫ 8.9k/22.7k [00:00<00:00, 35.3kit/s]
48.8%┣█████████████████▋                  ┫ 11.1k/22.7k [00:00<00:00, 36.6kit/s]
57.2%┣████████████████████▋               ┫ 13.0k/22.7k [00:00<00:00, 36.8kit/s]
65.9%┣███████████████████████▊            ┫ 15.0k/22.7k [00:00<00:00, 37.1kit/s]
74.1%┣██████████████████████████▊         ┫ 16.8k/22.7k [00:00<00:00, 37.1kit/s]
79.9%┣████████████████████████████▉       ┫ 18.1k/22.7k [00:01<00:00, 34.8kit/s]
90.1%┣████████████████████████████████▍   ┫ 20.4k/22.7k [00:01<00:00, 35.7kit/s]
100.0%┣█████████████████████


🚀 Starting VDCG...
Initial solution: f(x) = 0.737647


[ Info: Initial point x0 set on L1-ball boundary


...

  New point found: f(x) = 0.728908


[ Info: Interior point LP: τ = -6.77018838, Status = Interior


  New point found: f(x) = 0.727819
  ✅ Level 8 completed in 242.7844 sec | Best objective so far: 0.727501


[ Info: Interior point LP: τ = -28.60270843, Status = Interior


583.996868 seconds (21.64 M allocations: 412.821 GiB, 8.33% gc time, 0.23% compilation time: 13% of which was recompilation)

✅ VDCG Best Objective: 0.727501
✅ VDCG Total Time: 1039.1539 sec, Total Points: 122
📊 Running Standard CG...
📊 Standard CG Objective: 0.737647
🔍 Running Multi-start CG (time-limited)...
  Seed 23: f(x) = 0.737647, Time so far: 4.6699 sec, Points: 1
  Seed 24: f(x) = 0.730112, Time so far: 9.3839 sec, Points: 2
  Seed 25: f(x) = 0.729328, Time so far: 15.1119 sec, Points: 3
  Seed 26: f(x) = 0.728692, Time so far: 19.7839 sec, Points: 4
  Seed 27: f(x) = 0.730662, Time so far: 25.2588 sec, Points: 5
  Seed 28: f(x) = 0.728936, Time so far: 30.2728 sec, Points: 6
  Seed 29: f(x) = 0.892312, Time so far: 34.9485 sec, Points: 7
  Seed 30: f(x) = 0.760559, Time so far: 41.1081 sec, Points: 8
  Seed 31: f(x) = 0.728928, Time so far: 45.7503 sec, Points: 9
  Seed 32: f(x) = 0.728362, Time so far: 51.2281 sec, Points: 10
  Seed 33: f(x) = 0.730155, Time so far: 56.1643 